### Getting Data of Attractions in Barcelona

In [37]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver

### Install webdriver from: https://sites.google.com/a/chromium.org/chromedriver/downloads

###### Specify path to webdriver

In [218]:
browser = webdriver.Chrome("C:\\Users\Yen Leng\Downloads\chromedriver_win32\chromedriver.exe")

In [219]:
browser.get("https://www.tripadvisor.com.sg/Attractions-g187497-Activities-Barcelona_Catalonia.html#ATTRACTION_SORT_WRAPPER")

In [2]:
#check if there is next page
def hasNextPage(all_links):
    for link in all_links:
        if link.text == "Next":
            return True
        else:
            continue
    return False

In [220]:
soup = BeautifulSoup(browser.page_source, "lxml")

In [221]:
data = pd.DataFrame(columns = ['attraction', 'rating', 'lat', 'long'])
i = 0

repeat = True
page = soup.findAll("div", class_ = "unified pagination ")
all_links = page[0].findAll("a")

while repeat:
    #change div index for each attraction
    div = soup.findAll("div", class_ = "element_wrap")
    for i in range(0, len(div)):
        each = div[i]
        #Get rating
        imgs = each.find_all("img")
        for img in imgs:
            if img.get('alt') is None:
                continue
            elif "bubbles" in img.get('alt'):
                rating = img.get('alt')
            else:
                rating = False

        div_title = soup.findAll("div", class_ = "property_title")[i]
        #Get attraction name
        name = div_title.findAll("a")[0].text

        if rating == False:
            skip.append(name)
            continue
        else:
            #Get link to attraction
            inside = div_title.findAll("a")[0]['href']
            inside_link = "http://www.tripadvisor.com.sg" + inside
            browser.get(inside_link)
            soup_req = BeautifulSoup(browser.page_source,"lxml")

            #Get lat,long
            summary = soup_req.find_all('div', { "id" : "STATIC_MAP" })
            image = summary[0].find_all('img')
            a = str(image[0])
            m = re.search(("[0-9]+\.[0-9]*,[0-9]+\.[0-9]*"),a)
            lat = m.group(0).split(",")[0]
            long = m.group(0).split(",")[1]
            
            #Check progress
            print(i)
            print(name)
            
            #add into dataframe
            df = pd.DataFrame({'attraction':[name], 'rating':[rating], 'lat':[lat], 'long':[long]}, index = [i])
            data = data.append(df)
            i += 1

    if hasNextPage(all_links) == False:
        repeat = False
    else:
        next_page = page[0].findAll("a", string = "Next")[0]['href']
        new_link = "http://www.tripadvisor.com.sg" + next_page
        browser.get(new_link)
        soup = BeautifulSoup(browser.page_source,"lxml")
        page = soup.findAll("div", class_ = "unified pagination ")
        all_links = page[0].findAll("a")

0
Basilica of the Sagrada Familia
1
Casa Batllo
2
Palau de la Musica Orfeo Catala
3
Gothic Quarter (Barri Gotic)
6
Camp Nou
7
Mercat de Sant Josep de la Boqueria
8
The Basilica of Santa Maria del Mar
9
La Pedrera
10
Palau Guell
11
Park Guell
12
Sant Pau Recinte Modernista
13
Museu Nacional d'Art de Catalunya - MNAC
15
Passeig de Gracia
20
CosmoCaixa Barcelona
21
El Born
23
Catedral de Barcelona
24
Parc de la Ciutadella
25
Tibidabo
26
Ciutat Vella
28
Parc de Montjuic
0
Placa Espanya
1
Templo del Sagrado Corazon de Jesus
2
Fundacio Joan Miro
3
Museu d'Historia de Barcelona - MUHBA
4
Placa Sant Felip Neri
5
Barri de Gracia
7
Museo Picasso
8
Museu del Futbol Club Barcelona
10
Placa Reial
11
Barceloneta Beach
12
Reial Monestir de Santa Maria de Pedralbes
13
Casa Lleo i Morera
14
Las Ramblas
15
Palau Dalmases
16
El Puerto de Barcelona
17
Casa Amatller
18
Plaza de Cataluna
19
Hash Marihuana & Hemp Museum
20
La Barceloneta
21
Gran Teatre del Liceu
22
Parque del Laberinto de Horta
23
Museo d'Hi

In [223]:
data.to_csv("barcelona_attractions.csv")

In [224]:
browser.quit()